In [1]:
import pandas as pd
import numpy as np

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import random

from IPython.display import Markdown, display

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import cluster, datasets
from sklearn.cluster import KMeans

import pickle

In [2]:
#### Authentification

secrets_file = open("secrets.txt","r")
string = secrets_file.read()
#string.split('\n')

# Dictionary
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0: # excluding empty lines
        #           [first element:key]  [ second element:value  ]
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [3]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))


In [4]:
topsongs = pd.read_csv('topsongs_data.csv')
spotify_data = pd.read_csv('spotify_data.csv')

In [5]:
scaler = pickle.load(open('scaler_model.pkl','rb'))

In [6]:
kmeans = pickle.load(open('kmeans_model.pkl','rb'))

In [8]:
topsongs.columns

Index(['Unnamed: 0', 'title', 'artist'], dtype='object')

In [7]:
def features(track,artist):
    track_id = sp.search(q='artist' + artist + ' track' + track, type= 'track')
    uri = track_id['tracks']['items'][0]['i']
    features = sp.audio_features[uri]
    return features

In [10]:
new_song = input('Enter song: ').lower()
new_artist = input("Enter artist: ").lower()
print('\n')

#check if in topsongs
if new_song in list(topsongs['title']):
    recommend_hot = random.choice(list(topsongs['title']))
    while recommend_hot == new_song:
        recommend_hot = random.choice(list(topsongs['title']))
        print('Listen to: \n', recommendation_hot)
        
# Get spotify features
else:
    feature = features(new_song,new_artist)
    column = list(feature[0].keys())
    values = [list(feature[0].values())]
    df_new_song = pd.DataFrame(data=feature, columns = column)
    df_new_song = df_new_song.drop(['type','id','uri','track_href', 'analysis_url','time_signature'],axis=1)
    std_new_song = scaler.transform(df_new_song)
    new_cluster = kmeans.predict(std_new_song)
    df_cluster = list_clusters[list_cluster['cluster']==list(new_cluster)[0]]
    
    print('Listen to: \n', random.choice(list(df_cluster['song_and_artist'])))

Enter song: Fix you
Enter artist: 




KeyError: 'i'

In [ ]:
# Not in spotify function

In [ ]:
def check_song_in_spotify (song_searched):

    # Search for the song using the Spotify API
    results = sp.search(q='track:' + song_searched, type='track')

    # Check if there are multiple versions of the same song
    if results['tracks']['total'] > 1:
        print('Multiple versions of the song were found. Please specify the artist:')
        # Print all versions and the artist who played them
        for track in results['tracks']['items']:
            print(f"Track name: {track['name']}\nArtist name: {track['artists'][0]['name']}\n")

        # Ask to input just one of the artists
        artist_name = input('Enter the name of the artist: ')
        for track in results['tracks']['items']:
            if track['artists'][0]['name'].lower() == artist_name.lower():
                song_id = track['id']
                break
        else:
            print('No matching artist found. Exiting.')
            exit()
    elif results['tracks']['total'] == 1:
        song_id = results['tracks']['items'][0]['id']
    else:
        print('No results found for', song_title)
        exit()

    # Get audio features for the selected song
    song_features = sp.audio_features(song_id)[0]

    return song_features

In [ ]:
song_searched = input()

song_features =check_song_in_spotify (song_searched)

In [ ]:
type(song_features)

In [ ]:
song_features['danceability']

In [ ]:
tracks = []
track = {}

for key, value in song_features.items():
    track[key] = value

tracks.append(track)

tracks = pd.DataFrame(tracks)



tracks

tracks = tracks.select_dtypes(np.number)

tracks.shape

scaled_tracks = scaler.transform(tracks)
scaled_tracks

track_cluster = kmeans.predict(scaled_tracks)
track_cluster

In [ ]:
def check_song_in_spotify (song_searched):

    # Search for the song using the Spotify API
    results = sp.search(q='track:' + song_searched, type='track')

    # Check if there are multiple versions of the same song
    if results['tracks']['total'] > 1:
        print('Multiple versions of the song were found. Please specify the artist:')
        # Print all versions and the artist who played them
        for track in results['tracks']['items']:
            print(f"Track name: {track['name']}\nArtist name: {track['artists'][0]['name']}\n")

        # Ask to input just one of the artists
        artist_name = input('Enter the name of the artist: ')
        for track in results['tracks']['items']:
            if track['artists'][0]['name'].lower() == artist_name.lower():
                song_id = track['id']
                break
        else:
            print('No matching artist found. Exiting.')
            exit()
    elif results['tracks']['total'] == 1:
        song_id = results['tracks']['items'][0]['id']
    else:
        print('No results found for', song_title)
        exit()

    # Get audio features for the selected song
    song_features = sp.audio_features(song_id)[0]

    return song_features

In [ ]:
song_searched = input()

check_song_in_spotify (song_searched)

In [ ]:
def song_recommender(topsongs, spotify_data, song_searched):
    
    # Converting both input value and 'title' in lower case
    song_searched = song_searched.lower() 
    topsongs['title'] = topsongs['title'].str.lower()
    
    
    # ------------------ IF song in the input IS IN TOPSONGS -------------------------
    # ---------------- recommend random'hot' song fro topsong -----------------------
    
    if song_searched in topsongs['title'].values:
        # While song in input IS different from the recommended (output)
        while True:
            # Generating a random number to select another song from the list
            random_num = random.randint(0, len(topsongs)-1)
            # Picking the song 'title' with random_num index
            song_recommended_title = topsongs.iloc[random_num]['title']
            # Getting also the artist name 
            song_recommended_artist = topsongs.iloc[random_num]['artist']
            # If recommended song IS NOT the same as the searched one, stop the while
            if song_recommended_title != song_searched:
                break
        # Capitalizing first letters of 'title' (.title())
        song_recommended_title = song_recommended_title.title()
        # Joinning song and artist to display in the output
        song_recommended = f"{song_recommended_title} by {song_recommended_artist}"
        # Displaying output with both 'title' and 'artist' bigger and bolt style.
        display(Markdown(f"You should listen to: **{song_recommended}**!"))
    
    # ---------------- IF song in the input IS NOT IN TOPSONGS -----------------------
    else:
    
    # ------------------- Create CLUSTERS from SPOTIFY_DATA --------------------------
        
        # All numerical columns after 'uri' are the features to be taken
        features = spotify_data.select_dtypes(np.number)
        # Select X, but no target yet
        X = features
        # Scaling Data
        X_prep = StandardScaler().fit_transform(X)
        # Creating 8 clusters/segments (Kmeans)
        kmeans = KMeans(n_clusters=8, random_state=1234)
        kmeans.fit(X_prep)
        # Defining clusters:
        clusters = kmeans.predict(X_prep)
    
    # ---------------- IF song in the input IS IN SPOTIFY_DATA -----------------------
    
        # Converting both searched song and 'title' in spotify_data to lower case
        #song_searched = song_searched.lower()
        spotify_data['title'] = spotify_data['title'].str.lower()

        # Checking if the input song IS NOT IN the Spotify data
        if song_searched not in spotify_data['title'].values:
             # Selecting the features for the input song 
            # (filtering through boolean mask and selecting first row of filtered)
            song_features = features.loc[spotify_data['title']==song_searched].iloc[0]
            
            # Convert target_song to a numpy array and reshape to be a 2-dimensional array with one row
            song_features_arr = np.array(song_features).reshape(1, -1)
    
            # Scaling target_song values using the same X_prep scaler used for X data
            song_features_prep = StandardScaler().fit(X_prep).transform(song_features_arr)
    
            # Assigning the cluster using the same Kmeans fitted before:
            song_cluster = kmeans.predict(song_features_prep)
            
            
            # Recommending the most similar song from that cluster: 
        
            # Selecting all songs from the same cluster as song_searched (song_cluster)
            songs_in_cluster = spotify_data[kmeans.labels_ == song_cluster[0]]       ## Note(I)
    
            # Comparing song_searched with songs in same cluster using cosine_similarity
            from sklearn.metrics.pairwise import cosine_similarity                 ## Note (II)
    
            # Calculating cosine similarity between searched song and songs in cluster
            similarities = cosine_similarity(song_features_prep, songs_in_cluster.select_dtypes(np.number))
    
            # Finding the most similar song in the cluster
            most_similar_song_idx = np.argmax(similarities)                      ## Note (III)
    
            # Retrieving the most similar song from the cluster
            most_similar_song = songs_in_cluster.iloc[most_similar_song_idx]['title']
    
    
            # Recommending a random song from that cluster: 
        
            # Selecting a random song from that cluster
            random_song_idx = np.random.choice(np.where(kmeans.labels_ == song_cluster)[0])

            # Check random_song and most_similar_sonog are not the same. If so, repeat random choice.
            while spotify_data.iloc[random_song_idx]['title'] == most_similar_song:
                random_song_idx = np.random.choice(np.where(kmeans.labels_ == song_cluster)[0])

                # Getting title of random song
            random_song = spotify_data.iloc[random_song_idx]['title']


            display(Markdown(f"The song '**{song_searched.capitalize()}**' belongs to cluster '**{song_cluster[0]}**'.\n\n"
          f"The most similar song in the cluster is '**{most_similar_song.capitalize()}**'.\n\n"
          f"Here's a random song from the same cluster: '**{random_song.capitalize()}**'.")

        #else:
        #    return f"The song '**{song_searched}**' is not in our data."


In [ ]:
song_searched = input("Enter a song: ")
song_recommender(topsongs, spotify_data, song_searched)

In [ ]:
def song_recommender(topsongs, spotify_data, song_searched):
    
    # Converting both input value and 'title' in lower case
    song_searched = song_searched.lower() 
    topsongs['title'] = topsongs['title'].str.lower()
    
    # ------------------ IF song in the input IS IN TOPSONGS -------------------------
    
    # ------------------ If song in the input IS NOT IN TOPSONGS----------------------------
    
    else:
        
        # Defining clusters for audio features of spotify_data
        
        # ------------------ And song IS NOT IN SPOTIFY_DATA ------------------------------
        
        # Converting both searched song and 'title' in spotify_data to lower case
        song_searched = song_searched.lower()
        spotify_data['title'] = spotify_data['title'].str.lower()

        # Checking if the input song IS NOT IN the Spotify data
        if song_searched not in spotify_data['title'].values:
            
            # Searching specific song in Spotify
            new_song = sp.search(q='track'+ song_searched, type='track')
            
            # Checking if there are multiple versions of the same song, if so --> artist
            if new_song['tracks']['total'] > 1:
                print('Multiple versions of the song were found. Please specify the artist: \n')
                # Print list of versions and artist who played them
                for track in new_song['tracks']['items']:
                    print(f"Track name: {track['name']}\nArtist name: {track['artists'][0]['name']}\n")
                    
                # Ask to input just one of the artists/bands
                artist_name = input('Enter the name of the artist: ')
                
                # Checking which artist was chosen
                for track in new_song['tracks']['items']:
                    if track['artists'][0]['name'].lower() == artist_name.lower():
                        # ID of the song played by the artist selected
                        song_id = track['id']
                        break
                    else:
                        print('No matching artist found. Exiting.')
                        exit()
           
            # If there is just one version of that song
            elif new_song['tracks']['total'] == 1:
                song_id = new_song['tracks']['items'][0]['id']
            
            # If there is no coincidence at all
            else:
                print("No results found for ", song_title)
                exit()
                
            #Getting audio features for that song_id
            song_features = sp.audio_features(song_id)[0]
            
            # Selecting just numerical
            #song_features = audio_features.select_dtypes(np.number)
            
        
        # ------------------ but if song IS IN SPOTIFY_DATA ------------------------------ 
        
        else:
            # Extracting the features for the input song 
            # (filtering through boolean mask and selecting first row of filtered)
            song_features = features.loc[spotify_data['title']==song_searched].iloc[0]
           
    
    
        # Predicting/assigning a cluster for song_searched: --------------------------------
    
        # Solving dimensional issue and need for array:   
        
        # Exclude 'audio_features' key from song_features dictionary
        song_features_clean = {k: v for k, v in song_features.items() if k != 'audio_features'}

        # Convert target_song to a numpy array and reshape to be a 2-dimensional array with one row
        song_features_arr = np.array(list(song_features_clean.values())).reshape(1, -1)

        # Transforming target_song values using the same X_prep scaler used for X data
        song_features_prep = scaler.transform(song_features_arr)

        # Predicting / assigning the cluster using the same Kmeans fitted before:
        song_cluster = kmeans.predict(song_features_prep)
        
        
        
        
        # Convert target_song to a numpy array and reshape to be a 2-dimensional array with one row
        #song_features_arr = np.array(song_features).reshape(1, -1)
    
        # Scaling target_song values using the same X_prep scaler used for X data
        #song_features_prep = StandardScaler().fit(X_prep).transform(song_features_arr)
    
        # Predicting / assigning the cluster using the same Kmeans fitted before:
        #song_cluster = kmeans.predict(song_features_prep)
    
        
        # Recommending the most similar song from that cluster: -----------------------------
        
        # Selecting all songs from the same cluster as song_searched (song_cluster)
        songs_in_cluster = spotify_data[kmeans.labels_ == song_cluster[0]]
        
                    ## Note(I)
    
        # Comparing song_searched with songs in same cluster using cosine_similarity
        from sklearn.metrics.pairwise import cosine_similarity
        
                    ## Note (II)
    
        # Calculating cosine similarity between searched song and songs in cluster
        similarities = cosine_similarity(song_features_prep, songs_in_cluster.select_dtypes(np.number))
    
        # Finding the most similar song in the cluster
        most_similar_song_idx = np.argmax(similarities)
        
                    ## Note (III)
    
        # Retrieving the most similar song from the cluster
        most_similar_song = songs_in_cluster.iloc[most_similar_song_idx]['title']
    
    
        # Recommending another random song from that cluster: -------------------------------
        
        # Selecting a random song from that cluster
        random_song_idx = np.random.choice(np.where(kmeans.labels_ == song_cluster)[0])

        # Check random_song and most_similar_sonog are not the same. If so, repeat random choice.
        while spotify_data.iloc[random_song_idx]['title'] == most_similar_song:
            random_song_idx = np.random.choice(np.where(kmeans.labels_ == song_cluster)[0])

            # Getting title of random song
            random_song = spotify_data.iloc[random_song_idx]['title']

    
        display(Markdown(f"The song '**{song_searched.capitalize()}**' belongs to cluster '**{song_cluster[0]}**'.\n\n"
                  f"The most similar song in the cluster is '**{most_similar_song.capitalize()}**'.\n\n"))
                 # f"Here's a random song from the same cluster: '**{random_song.capitalize()}**'."))

 
    
        #else:
    
            #return f"The song '**{song_searched}**' is not in our data."

In [ ]:
song_searched = input("Enter a song: ")
song_recommender(topsongs, spotify_data, song_searched)